# Couple Candlesticks - Strategy - Backtesting 

### Import Library

In [73]:
import numpy as np
import pandas as pd
import numpy as np
import pandas_ta as ta
from backtesting.backtesting import Backtest, Strategy
from backtesting._plotting import set_bokeh_output
set_bokeh_output(notebook=False)

import matplotlib.pyplot as plt
plt.rcParams['figure.figsize'] = [12, 6]
plt.rcParams['figure.dpi'] = 120
import warnings
warnings.filterwarnings('ignore')

### Load Price Data

In [74]:
import os
from pathlib import Path
notebook_path = os.getcwd()
algo_dir = Path(notebook_path).parent.parent
csv_file = str(algo_dir) + '/vn-stock-data/VN30ps/VN30F1M_5minutes.csv'
is_file = os.path.isfile(csv_file)
if is_file:
    dataset = pd.read_csv(csv_file, index_col='Date', parse_dates=True)
else:
    print('remote')
    dataset = pd.read_csv("https://raw.githubusercontent.com/zuongthaotn/vn-stock-data/main/VN30ps/VN30F1M_5minutes.csv", index_col='Date', parse_dates=True)

In [75]:
data = dataset.copy()

In [76]:
# data = data[(data.index > '2020-11-01 00:00:00') & (data.index < '2024-10-01 00:00:00')]
data = data[data.index > '2020-11-01 00:00:00']

In [77]:
data

,Open,High,Low,Close,Volume
Date,,,,,
2020-11-02 09:00:00,900.1,900.2,899.3,900.1,1910
2020-11-02 09:05:00,900.2,900.2,898.7,899.4,1670
2020-11-02 09:10:00,899.5,900.0,899.0,899.5,1329
2020-11-02 09:15:00,899.4,899.5,898.2,898.6,1722
2020-11-02 09:20:00,898.5,898.6,896.5,898.2,2939
...,...,...,...,...,...
2024-11-08 14:15:00,1321.0,1321.8,1318.3,1320.7,9834
2024-11-08 14:20:00,1320.5,1322.3,1320.3,1320.9,5353
2024-11-08 14:25:00,1321.0,1325.0,1320.8,1324.6,8257


In [78]:
def set_condition_1(r):
    cond = ''
    if r['Open'] > r['Close'] and r['Close'] >= r['Low'] + 0.1:
        # Do va co bong nen duoi
        cond = 'short'
    elif r['Open'] < r['Close'] and r['Close'] <= r['High'] - 0.1:
        # Xanh va co bong nen tren
        cond = 'long'
    return cond

def set_condition_2(r):
    cond = ''
    if r['Open'] > r['Close'] and r['Close'] == r['Low'] and r['Low'] < r['prev_low']:
        # Do va khong co bong nen duoi
        cond = 'short'
    elif r['Open'] < r['Close'] and r['Close'] == r['High']  and r['High'] > r['prev_high']:
        # Xanh va khong co bong nen tren
        cond = 'long'
    return cond

def get_signal(r):
    signal = ''
    if r['condition_1'] == 'short' and r['condition_2'] == 'short':
        signal = 'short'
    elif r['condition_1'] == 'long' and r['condition_2'] == 'long':
        signal = 'long'
    return signal

In [79]:
def prepare_data(data):
    data['prev_low'] = data['Low'].shift(1)
    data['prev_high'] = data['High'].shift(1)
    data['max_5'] = data['High'].rolling(5).max()
    data['min_5'] = data['Low'].rolling(5).min()
    data['condition_1'] = data.apply(lambda r: set_condition_1(r), axis=1)
    data['condition_1'] = data['condition_1'].shift(1)
    data['condition_2'] = data.apply(lambda r: set_condition_2(r), axis=1)
    data['signal'] = data.apply(lambda r: get_signal(r), axis=1)
    return data

In [80]:
class CoupleCandlesticks(Strategy):
    max_sl = 4.5
    def init(self):
        super().init()

    def next(self):
        super().next()
        _time = self.data.index
        current_time = _time[-1]
        if current_time.hour == 14 and current_time.minute >= 25:
            if self.position.is_long or self.position.is_short:
                self.position.close()
                return
                
        if current_time.hour == 14 and current_time.minute >= 30:
            return

        close_price = self.data.Close[-1]
        signal = self.data.signal[-1]
        if signal == 'long':
            if self.position.is_short:
                self.position.close()
            buy_price = close_price
            min_5 = self.data.min_5[-1]
            tmp_risk = buy_price - min_5
            risk = min(tmp_risk, self.max_sl)
            sl = buy_price - risk
            self.buy(size=1, sl=sl)
        elif signal == 'short':
            if self.position.is_long:
                self.position.close()
            sell_price = close_price
            max_5 = self.data.max_5[-1]
            tmp_risk = max_5 - sell_price
            risk = min(tmp_risk, self.max_sl)
            sl = sell_price + risk
            self.sell(size=1, sl=sl)

In [81]:
prepared_data = prepare_data(data)
prepared_data.dropna(inplace=True)

In [82]:
# prepared_data[(prepared_data.index > '2024-11-06 00:00:00') & (prepared_data.index < '2024-11-06 15:00:00')]

In [83]:
bt = Backtest(prepared_data, CoupleCandlesticks, commission=.0003, exclusive_orders=False)
stats = bt.run()
# bt.plot()
print(stats)

Start                     2020-11-02 09:20:00
End                       2024-11-08 14:45:00
Duration                   1467 days 05:25:00
Exposure Time [%]                   41.420846
Equity Final [$]                  11017.35054
Equity Peak [$]                    11122.8076
Return [%]                          10.173505
Buy & Hold Return [%]               47.405923
Return (Ann.) [%]                    2.471323
Volatility (Ann.) [%]                 2.20016
Sharpe Ratio                         1.123247
Sortino Ratio                        3.120887
Calmar Ratio                         1.021201
Max. Drawdown [%]                   -2.420017
Avg. Drawdown [%]                   -0.158485
Max. Drawdown Duration      243 days 00:10:00
Avg. Drawdown Duration       10 days 08:36:00
# Trades                                 2132
Win Rate [%]                        34.099437
Best Trade [%]                       6.822596
Worst Trade [%]                     -0.920813
Avg. Trade [%]                    

In [84]:
stats['_trades']

,Size,EntryBar,ExitBar,EntryPrice,ExitPrice,PnL,ReturnPct,EntryTime,ExitTime,Tag,Duration
0,-1,10,11,899.63003,901.4,-1.76997,-0.001967,2020-11-02 10:10:00,2020-11-02 10:15:00,None,0 days 00:05:00
1,1,45,46,906.97201,904.0,-2.97201,-0.003277,2020-11-02 14:30:00,2020-11-02 14:45:00,None,0 days 00:15:00
2,-1,52,65,907.92754,909.4,-1.47246,-0.001622,2020-11-03 09:25:00,2020-11-03 10:30:00,None,0 days 01:05:00
3,-1,106,111,907.02781,908.8,-1.77219,-0.001954,2020-11-04 09:40:00,2020-11-04 10:05:00,None,0 days 00:25:00
4,1,123,126,914.27420,912.3,-1.97420,-0.002159,2020-11-04 11:05:00,2020-11-04 11:20:00,None,0 days 00:15:00
...,...,...,...,...,...,...,...,...,...,...,...
2127,-1,51063,51068,1319.60400,1320.2,-0.59600,-0.000452,2024-11-05 14:05:00,2024-11-05 14:30:00,None,0 days 00:25:00
2128,-1,51089,51097,1326.00208,1327.4,-1.39792,-0.001054,2024-11-06 10:35:00,2024-11-06 11:15:00,None,0 days 00:40:00
2129,1,51135,51145,1338.50143,1339.2,0.69857,0.000522,2024-11-07 10:10:00,2024-11-07 11:00:00,None,0 days 00:50:00
2130,-1,51145,51155,1338.79824,1340.5,-1.70176,-0.001271,2024-11-07 11:00:00,2024-11-07 13:15:00,None,0 days 02:15:00


In [85]:
stats['_trades']['PnL'].sum()

1017.3505400000694

### Optimize

In [86]:
# trades = stats['_trades']
# max_d = trades['Duration'].max()
# trades[trades.Duration == max_d]

In [87]:
max_sl_params = [x / 10.0 for x in range(20, 60, 5)]
optimize_stats, heatmap = bt.optimize(max_sl=max_sl_params,
                    maximize='Return [%]',
                    random_state=0,
                    return_heatmap=True)

In [88]:
optimize_stats

Start                     2020-11-02 09:20:00
End                       2024-11-08 14:45:00
Duration                   1467 days 05:25:00
Exposure Time [%]                   41.420846
Equity Final [$]                  11017.35054
Equity Peak [$]                    11122.8076
Return [%]                          10.173505
Buy & Hold Return [%]               47.405923
Return (Ann.) [%]                    2.471323
Volatility (Ann.) [%]                 2.20016
Sharpe Ratio                         1.123247
Sortino Ratio                        3.120887
Calmar Ratio                         1.021201
Max. Drawdown [%]                   -2.420017
Avg. Drawdown [%]                   -0.158485
Max. Drawdown Duration      243 days 00:10:00
Avg. Drawdown Duration       10 days 08:36:00
# Trades                                 2132
Win Rate [%]                        34.099437
Best Trade [%]                       6.822596
Worst Trade [%]                     -0.920813
Avg. Trade [%]                    

In [89]:
heatmap

max_sl
2.0        8.247032
2.5        9.542356
3.0        9.572581
3.5        9.955394
4.0        9.954550
4.5       10.173505
5.0        9.806795
5.5        8.894795
Name: Return [%], dtype: float64